# torch geometric + skorch @ CORA dataset

In [1]:
!date

Fr 22. Jul 18:52:31 CEST 2022


This is an example for how to use skorch with [torch geometric](https://pytorch-geometric.readthedocs.io/). The code is based on the [introduction example](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html) but modified to have a proper train/valid/test split. This example is showcasing a quite small data set that does not need to employ batching to be trained efficiently. How to do batching with skorch + torch geometric will not be handled here since it is non-trivial and quite dataset specific - if you need this and are stuck, feel free to open [an issue](https://github.com/skorch-dev/skorch/issues) so that we can support you the best we can.

Dependencies of this notebook besides skorch base installation:

It is recommended to install the dependencies [as documented by pytorch geometric](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html).

---

In [2]:
import skorch
import torch

### Data Loading

In [3]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [4]:
dataset.data, dataset.num_classes

(Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708]),
 7)

In order to use pytorch geometric / the cora dataset with skorch
we need to address the following things:
    
1. graph convolutions cannot handle missing nodes (=> splitting node attributes but keeping edge_index intact will lead to errors)
2. cora dataset has different attributes for the different split masks (i.e. `train_mask`, `val_mask`, `test_mask`)
3. skorch expects to have (X, y) pairs for classification tasks

To deal with (1) we will split the data into three datasets, creating three sub-graphs in the process; these complete sub-graphs can then be convolved over without errors. 
We use the masks mentioned in (2) to identify the nodes and edges of the subgraphs.

(3) will be handled by specifying our own `XYDataset` which will just have length 1 and return the dataset and the respective y values. We will therefore basically simulate a `batch_size=1` scenario.

In [5]:
from torch_geometric.data import Data

# simulating batch_size=1 by returning the whole dataset and the
# y-values. this way, the data loader can iterate over the 'batches'
# and produce X/y values for us.
class XYDataset(torch.utils.data.Dataset):
    def __init__(self, data: Data, y: torch.tensor):
        self.data = data
        self.y = y
        
    def __len__(self):
        return 1
        
    def __getitem__(self, i):
        return self.data, self.y

### Data Splitting

Split the graph into train, validation and test sub-graphs.
This ensures that there will be no leakage between steps when we apply graph
convolution operators on the graph since each split has its own sub-graph.

We use `relabel_nodes=True` to make the node indices in the edge tensor 
zero-based for each sub-graph. If we would not do this the node subsets
(now zero-based after applying the mask) would not match the indices in the
edge tensor.

In [6]:
from torch_geometric.utils import subgraph

data = dataset[0]

edge_index_train, _ = subgraph(
    subset=data.train_mask, 
    edge_index=data.edge_index, 
    relabel_nodes=True
)
ds_train = XYDataset(
    Data(x=data.x[data.train_mask], edge_index=edge_index_train),
    data.y[data.train_mask],
)

edge_index_valid, _ = subgraph(
    subset=data.val_mask, 
    edge_index=data.edge_index, 
    relabel_nodes=True
)
ds_valid = XYDataset(
    Data(x=data.x[data.val_mask], edge_index=edge_index_valid),
    data.y[data.val_mask],
)

edge_index_test, _ = subgraph(
    subset=data.test_mask, 
    edge_index=data.edge_index, 
    relabel_nodes=True
)
ds_test = XYDataset(
    Data(x=data.x[data.test_mask], edge_index=edge_index_test),
    data.y[data.test_mask],
)

### Data Feeding

Our "batch" consists of the whole dataset so if we unpack the
batch into `(X, y)` we will have `X = Data(...)` and `y = [y_true]`.
The `DataLoader` does not modify `X` but `y` gets a new batch dimension.
This will lead to a shape mismatch as `y.shape` would then be `(1, #num_samples)`. Therefore, we need our own loader that strips the first dimension to 
match the predicted `y` and the labelled `y` in length.

Note: It is possible to avoid this by stripping this dimension by overriding `get_loss` in the `NeuralNet` class. For brevity we won't do this in this example. It is possible to use [one of the many `DataLoader` classes](https://pytorch-geometric.readthedocs.io/en/latest/modules/loader.html) provided by torch geometric using the approach outlined below (just base the `RawDataloader` on one of the other classes) - chances are, though, that if you are doing this you need to deal with batching anyway which is a topic that is not handled here since it is not trivial.

In [7]:
from torch_geometric.loader import DataLoader

class RawLoader(DataLoader):
    def __iter__(self):
        it = super().__iter__()
        for X, y in it:
            yield X, y[0]

### Modelling

This is the CORA example module as seen in the [torch geometric introduction](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html).

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):        
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.softmax(x, dim=1)

### Fitting

In [9]:
from skorch.helper import predefined_split

torch.manual_seed(42)

net = skorch.NeuralNetClassifier(
    module=GCN,
    lr=0.1,
    optimizer__weight_decay=5e-4,
    max_epochs=200,
    train_split=skorch.helper.predefined_split(ds_valid),
    batch_size=1,
    iterator_train=RawLoader,
    iterator_valid=RawLoader,
)

In [10]:
net.fit(ds_train, None)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.9724       0.1680        1.9398  0.0074
      2        1.9625       0.1740        1.9376  0.0052
      3        1.9327       0.1720        1.9342  0.0043
      4        1.9321       0.1760        1.9324  0.0045
      5        1.9142       0.1800        1.9307  0.0037
      6        1.8923       0.1800        1.9290  0.0036
      7        1.8848       0.1880        1.9269  0.0042
      8        1.8936       0.1920        1.9247  0.0042
      9        1.8783       0.1960        1.9219  0.0044
     10        1.8737       0.2040        1.9192  0.0044
     11        1.8542       0.2060        1.9176  0.0042
     12        1.8489       0.2100        1.9156  0.0042
     13        1.8314       0.2120        1.9121  0.0074
     14        1.8334       0.2220        1.9100  0.0055
     15        1.8041       0.2240        1.9085  0.0081
     16        1.8089       0.2

    107        0.9986       0.5320        1.5732  0.0047
    108        0.9234       0.5320        1.5692  0.0070
    109        0.9704       0.5300        1.5642  0.0065
    110        1.0256       0.5300        1.5621  0.0087
    111        0.9590       0.5240        1.5585  0.0074
    112        1.0168       0.5300        1.5565  0.0057
    113        0.9994       0.5320        1.5534  0.0073
    114        0.9635       0.5320        1.5492  0.0082
    115        0.9872       0.5340        1.5452  0.0046
    116        0.9749       0.5340        1.5411  0.0092
    117        0.9667       0.5340        1.5392  0.0048
    118        0.8757       0.5300        1.5351  0.0088
    119        0.9306       0.5340        1.5340  0.0072
    120        0.8284       0.5380        1.5300  0.0085
    121        0.8389       0.5400        1.5254  0.0086
    122        0.9347       0.5440        1.5226  0.0082
    123        0.8502       0.5340        1.5207  0.0064
    124        0.8519       0.5

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=GCN(
    (conv1): GCNConv(1433, 16)
    (conv2): GCNConv(16, 7)
  ),
)

### Evaluation

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
accuracy_score(ds_test.y, net.predict(ds_test))

0.682

In conclusion this example showed you how to use a basic data graph dataset using pytorch geometric in conjunction with skorch. The final test score is lower than the ~80% accuracy in the [introduction example](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html) which can be explained by the reduced leakage between train and validation sets due to our splitting the data into subgraphs beforehand.

The model is now incorporated into the sklearn world (as you could already see, you can simply use sklearn metrics to evaluate the model). Thus, tools like grid and random search are available to you and it is easily possible to include a graph neural net as a feature transformer in your next ML pipeline!